In [15]:
from joblib import Parallel, delayed
from sim.dy import Model
from sim import load_inputs
import json
import pandas as pd
import matplotlib.pylab as plt
import os

In [3]:
with open('data/test_priors.json', 'r') as f:
    prior = json.load(f)

print(f'Prior size: {len(prior)}')
    
inputs = load_inputs('data/pars.json')

m = Model(inputs, year0=1970)

Prior size: 200


In [9]:
def fn(i, p0, model):
    _, ms, _ = model.simulate_to_fit(p0)
    ms = pd.DataFrame(ms).assign(Key = i)
    return ms

In [12]:
with Parallel(n_jobs=5, verbose=4) as parallel:
    mss = parallel(delayed(fn)(i, p, m) for i, p in enumerate(prior))
    
mss = pd.concat(mss)
mss.head()

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    2.8s
[Parallel(n_jobs=5)]: Done  88 tasks      | elapsed:   16.6s
[Parallel(n_jobs=5)]: Done 200 out of 200 | elapsed:   37.4s finished


,Time,Pop,MorR,PropComorb,Pop_RiskLo,MorR_RiskLo,Pop_RiskHi,MorR_RiskHi,Prev,ARTI,...,FP_Pcf,FP_Acf,PPV,PPV_Pcf,PPV_Acf,N_ACF_Reached,N_ACF_TB_Reached,N_ACF_NonTB_Reached,R_ACF_Reached,Key
0,2006.0,1.877611e+06,0.001553,0.055120,1.774117e+06,0.001193,103493.814270,0.007727,0.007217,0.028441,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
1,2012.0,2.085482e+06,0.001531,0.055120,1.970530e+06,0.001176,114951.677555,0.007624,0.007116,0.028105,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
2,2018.0,2.316367e+06,0.001521,0.055120,2.188689e+06,0.001167,127678.047881,0.007582,0.007068,0.027988,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
0,2006.0,1.877611e+06,0.001574,0.045318,1.792522e+06,0.001341,85088.770225,0.006486,0.007318,0.034145,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1
1,2012.0,2.085482e+06,0.001541,0.045318,1.990973e+06,0.001313,94508.999833,0.006353,0.007162,0.033587,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1


In [16]:
out_path = 'out/dy_free'

os.makedirs(out_path, exist_ok=True)

mss.to_csv(f'{out_path}/Runs_Prior.csv')